## Importing Libraries

In [ ]:
import os
import pandas as pd
import json
import numpy as np

In [ ]:
# %pip install git+https://github.com/JustAnotherArchivist/snscrape.git
# import snscrape.modules.twitter as sntwitter

In [ ]:
%pip install googletrans==4.0.0-rc1
%pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Checking version of googletrans that works
%pip show googletrans

Name: googletrans
Version: 4.0.0rc1
Summary: Free Google Translate API for Python. Translates totally free of charge.
Home-page: https://github.com/ssut/py-googletrans
Author: SuHun Han
Author-email: ssut@ssut.me
License: MIT
Location: /usr/local/lib/python3.8/dist-packages
Requires: httpx
Required-by: 


In [ ]:
import googletrans
from googletrans import Translator
from langdetect import detect

## Scraping Data

In [ ]:
# # 
# # Creating list to append tweet data to
# attributes_container = []

# # Using TwitterSearchScraper to scrape data and append tweets to list
# for i,tweet in enumerate(sntwitter.TwitterSearchScraper('pakistan politics since:2022-06-01 until:2022-12-31').get_items()):

#     dt = (str(tweet.date)[0:10])
#     attributes_container.append([tweet.user.username, dt, tweet.likeCount, tweet.sourceLabel, tweet.content, tweet.retweetCount, tweet.likeCount])
    



In [ ]:
# # save to a json file
# temp_j = json.dumps(attributes_container)

# # Writing to sample.json
# with open("/content/drive/MyDrive/Personal Projects/Pak Election Prediction/data/tweets_data.json", "w") as outfile:
#     outfile.write(temp_j)

# Creating and Saving into a Table

In [ ]:
# Creating a dataframe to load the list
# tweets_df = pd.DataFrame(attributes_container, columns=["User", "Date Created", "Number of Likes", "Source", "Tweet","Retweets", "Likes"])

In [ ]:
# tweets_df.to_csv('/content/drive/MyDrive/Personal Projects/Pak Election Prediction/data/tweets_data.csv', index=False)

# Importing the Scraped Dataset

In [ ]:
tweets_df = pd.read_csv('/content/drive/MyDrive/Personal Projects/Pak Election Prediction/data/tweets_data.csv')

In [ ]:
tweets_df

,User,Date Created,Number of Likes,Source,Tweet,Retweets,Likes
0,MEDICOSEFEVER,12/30/2022,0.0,Twitter for Android,Past politics experience can bring prosperity ...,0.0,0.0
1,Zeenat84065967,12/30/2022,0.0,Twitter for Android,پاکستان کی سیاست کے بارے میں 2022 کے اختتام تک...,0.0,0.0
2,linkarif,12/30/2022,0.0,Twitter for iPhone,"@OfficialDGISPR Stop playing dirty politics, i...",0.0,0.0
3,News_Pakistan10,12/30/2022,0.0,Twitter Web App,See Donald Trump's tax returns | CNN Politics\...,0.0,0.0
4,DunyaNews,12/30/2022,181.0,DunyaTV,بلوچستان کے چار اراکین اسمبلی نے پیپلز پارٹی م...,14.0,181.0
...,...,...,...,...,...,...,...
76947,ArchiAsim,6/1/2022,0.0,Twitter for iPhone,Pakistan and it’s politics going through very ...,0.0,0.0
76948,RashidA25299212,6/1/2022,0.0,Twitter for Android,@RashidA25299212 \nPOLITICS OF PAKISTAN - NA F...,0.0,0.0
76949,SAFFAA007,6/1/2022,1.0,Twitter for Android,Pakistan's politics.... https://t.co/ZapqpOJxCC,0.0,1.0
76950,David_Rosenfild,6/1/2022,1.0,Twitter for Android,@FatimaFarooqAli Kahe ki predictions .... Jitn...,0.0,1.0


## Data Overview

In [ ]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76952 entries, 0 to 76951
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   User             76952 non-null  object 
 1   Date Created     76950 non-null  object 
 2   Number of Likes  76950 non-null  float64
 3   Source           76949 non-null  object 
 4   Tweet            76949 non-null  object 
 5   Retweets         76948 non-null  float64
 6   Likes            76948 non-null  float64
dtypes: float64(3), object(4)
memory usage: 4.1+ MB


## Removing Rows with Null Values

In [ ]:
tweets_df = tweets_df.dropna()

In [ ]:
## Reseting index due to removed rows
tweets_df = tweets_df.reset_index(drop=True)

In [ ]:
data = tweets_df.Tweet

Two libraries were loaded namely 'googletrans'(https://stackabuse.com/text-translation-with-google-translate-api-in-python/) and 'langdetec' (https://pypi.org/project/langdetect/) both of which are made by google. Since a lot of tweets may be in 'urdu' so correctly identify the sentiments, it was important to translate them for further analysis rather than removing them.

## Cleaning and Translating

### Translating to english

USE THE INDICES IN 'NOLANG' TO REMOVE THE ROWS IN THE TABLE WHICH WERE NOT TRANSLATED

In [ ]:
nolang = []
for i,tweet in enumerate(data):
  try:
    if detect(data[i]) != "en":
      data[i] = Translator().translate(data[i]).text
  except:
    nolang.append(i)
    data.pop(i)
data = data.reset_index(drop=True)

<ipython-input-16-8f58316ff50b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = Translator().translate(data[i]).text


## Cleaning the text

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import string

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Following steps are carried out:

1- Removing hashtags, @ etc

2- Tokenizing sentences

3- Removing Stopwords

4- Removing punctuations from words

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
lemmatizer = WordNetLemmatizer()

# function to clean the text following the above mention steps
def clean_tweet(tweet):
  tweet = tweet.lower()
  tweet_tok = nltk.TweetTokenizer().tokenize(tweet)

  ## lemmatizer is basic and only considering the words as nouns, can be changed to advanced lemmatizer from wordblob etc
  tweet_lem = []
  for tokens in tweet_tok:
    tweet_lem.append(lemmatizer.lemmatize(tokens))

  ## Removing Links and punctuations in any words
  tweet_tok = [ t for t in tweet_lem
            if not t.startswith("http://")
            and not t.startswith("https://")]

  tweet_tok = [t.lstrip('@') for t in tweet_tok]
  tweet_tok = [t.lstrip('#') for t in tweet_tok]

  cleaned = []
  for t in tweet_tok:
      cleaned.append(
          t.translate(str.maketrans('', '', string.punctuation)).strip())
  tweet_tok = [t for t in cleaned if t]

  cleaned_tweet = ' '.join(tweet_tok)
  return cleaned_tweet

  



## Positive / Negative Tweets


In [ ]:
!pip install textblob
!pip install tweepy
!pip install pycountry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from textblob import TextBlob
import tweepy
import pycountry

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
pos_tweets = []
neg_tweets = []
nuet_tweets = []
sa = SentimentIntensityAnalyzer()

In [ ]:
# Cleaning the tweets and overwriting the current tweets with the cleaned version
# Also marking the tweets as postive and negative in te same step to save memory
for i in range(len(data)):
  temp = clean_tweet(data[i])
  sc = (sa.polarity_scores(temp))['compound']
  data[i] = temp
  if sc > 0:
    pos_tweets.append(i)
  elif sc < 0:
    neg_tweets.append(i)
  else:
    nuet_tweets.append(i)

In [ ]:
len(pos_tweets) , len(neg_tweets), len(nuet_tweets)

(30531, 27271, 11276)

In [ ]:
pos_tweets[0:10]

[1, 2, 4, 5, 7, 9, 13, 19, 21, 29]

## Modifying the Initial table with clean texts and Sentiment Labels

In [ ]:
tweets_df.drop(nolang, axis = 0, inplace = True)
## Reseting index due to removed rows
tweets_df = tweets_df.reset_index(drop=True)

In [ ]:
tweets_df['sentim']= 0
for i in range(len(tweets_df.sentim)):
  if i in pos_tweets:
    tweets_df.sentim[i] = 1
  elif i in neg_tweets:
    tweets_df.sentim[i] = -1
  else:
    tweets_df.sentim[i] = 0

<ipython-input-26-69131eb93163>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df.sentim[i] = -1
<ipython-input-26-69131eb93163>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df.sentim[i] = 1
<ipython-input-26-69131eb93163>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df.sentim[i] = 0


In [ ]:
tweets_df['cleaned_tweets'] = data

In [ ]:
len(data)

69078

In [ ]:
tweets_df

,User,Date Created,Number of Likes,Source,Tweet,Retweets,Likes,sentim,cleaned_tweets
0,MEDICOSEFEVER,12/30/2022,0.0,Twitter for Android,Past politics experience can bring prosperity ...,0.0,0.0,-1,past politics experience can bring prosperity ...
1,Zeenat84065967,12/30/2022,0.0,Twitter for Android,A comprehensive analysis of Arshad Bhatti by t...,0.0,0.0,1,a comprehensive analysis of arshad bhatti by t...
2,linkarif,12/30/2022,0.0,Twitter for iPhone,"@OfficialDGISPR Stop playing dirty politics, i...",0.0,0.0,1,officialdgispr stop playing dirty politics it ...
3,News_Pakistan10,12/30/2022,0.0,Twitter Web App,See Donald Trump's tax returns | CNN Politics\...,0.0,0.0,0,see donald trumps tax return cnn politics
4,DunyaNews,12/30/2022,181.0,DunyaTV,Four members of Balochistan join the People's ...,14.0,181.0,1,four member of balochistan join the peoples party
...,...,...,...,...,...,...,...,...,...
69073,BashirChaudhr10,6/1/2022,0.0,Twitter for iPad,Pakistan politics….establishment should make s...,0.0,0.0,1,pakistan politics … establishment should make ...
69074,ArchiAsim,6/1/2022,0.0,Twitter for iPhone,Pakistan and it’s politics going through very ...,0.0,0.0,-1,pakistan and it ’ s politics going through ver...
69075,RashidA25299212,6/1/2022,0.0,Twitter for Android,@RashidA25299212 \nPOLITICS OF PAKISTAN - NA F...,0.0,0.0,1,rashida25299212 politics of pakistan na forum ...
69076,SAFFAA007,6/1/2022,1.0,Twitter for Android,Pakistan's politics.... https://t.co/ZapqpOJxCC,0.0,1.0,0,pakistans politics


In [ ]:
# tweets_df.to_csv('/content/drive/MyDrive/Personal Projects/Pak Election Prediction/data/tweets_dataset_final.csv', index=False)

## Data Overview

In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/Personal Projects/Pak Election Prediction/data/tweets_dataset_final.csv')

In [ ]:
final_df.drop('Number of Likes', axis = 1, inplace = True)
final_df.head()

,User,Date Created,Source,Tweet,Retweets,Likes,sentim,cleaned_tweets
0,MEDICOSEFEVER,12/30/2022,Twitter for Android,Past politics experience can bring prosperity ...,0.0,0.0,-1,past politics experience can bring prosperity ...
1,Zeenat84065967,12/30/2022,Twitter for Android,A comprehensive analysis of Arshad Bhatti by t...,0.0,0.0,1,a comprehensive analysis of arshad bhatti by t...
2,linkarif,12/30/2022,Twitter for iPhone,"@OfficialDGISPR Stop playing dirty politics, i...",0.0,0.0,1,officialdgispr stop playing dirty politics it ...
3,News_Pakistan10,12/30/2022,Twitter Web App,See Donald Trump's tax returns | CNN Politics\...,0.0,0.0,0,see donald trumps tax return cnn politics
4,DunyaNews,12/30/2022,DunyaTV,Four members of Balochistan join the People's ...,14.0,181.0,1,four member of balochistan join the peoples party


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69078 entries, 0 to 69077
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   User            69078 non-null  object 
 1   Date Created    69078 non-null  object 
 2   Source          69078 non-null  object 
 3   Tweet           69078 non-null  object 
 4   Retweets        69078 non-null  float64
 5   Likes           69078 non-null  float64
 6   sentim          69078 non-null  int64  
 7   cleaned_tweets  69071 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 4.2+ MB


In [ ]:
final_df = final_df.dropna()
## Reseting index due to removed rows
final_df = final_df.reset_index(drop=True)

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69071 entries, 0 to 69070
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   User            69071 non-null  object 
 1   Date Created    69071 non-null  object 
 2   Source          69071 non-null  object 
 3   Tweet           69071 non-null  object 
 4   Retweets        69071 non-null  float64
 5   Likes           69071 non-null  float64
 6   sentim          69071 non-null  int64  
 7   cleaned_tweets  69071 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 4.2+ MB


In [ ]:
final_df.to_csv('/content/drive/MyDrive/Personal Projects/Pak Election Prediction/data/tweets_dataset_final.csv', index=False)